# IVI Data Exploration and Cleaning Pipeline

This notebook covers:
1. Loading the raw SAS datasets
2. Initial exploration (shape, dtypes, samples)
3. Missing value analysis
4. Outlier detection
5. Data aggregation at contract level
6. Creating a cleaned, unified dataset

In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

# Define paths
DATA_DIR = Path('../data/KAU-Bupa')
OUTPUT_DIR = Path('../data/processed')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print('Libraries loaded successfully.')

Libraries loaded successfully.


## 1. Load Raw Datasets

In [ ]:
# Load SAS datasets
print('Loading datasets...')

# Load members data
df_member = pd.read_sas(DATA_DIR / 'sampled_member.sas7bdat', encoding='latin1')
print(f'Members loaded: {df_member.shape}')

# Load claims data
df_claims = pd.read_sas(DATA_DIR / 'sampled_claims.sas7bdat', encoding='latin1')
print(f'Claims loaded: {df_claims.shape}')

# Load calls data
df_calls = pd.read_sas(DATA_DIR / 'sampled_calls.sas7bdat', encoding='latin1')
print(f'Calls loaded: {df_calls.shape}')

# Load preauth data
df_preauth = pd.read_sas(DATA_DIR / 'sampled_preauth.sas7bdat', encoding='latin1')
print(f'Preauth loaded: {df_preauth.shape}')

print('\nAll datasets loaded successfully.')

Loading datasets...
Members loaded: (4263550, 9)


In [ ]:
# Load provider info (Excel)
df_provider = pd.read_excel(DATA_DIR / 'Provider_Info.xlsx')
print(f'Provider Info loaded: {df_provider.shape}')
df_provider.head()

## 2. Initial Data Exploration

In [ ]:
# Function to display dataset info
def explore_dataset(df, name):
    print('=' * 60)
    print(f'Dataset: {name}')
    print('=' * 60)
    print(f'Shape: {df.shape[0]:,} rows x {df.shape[1]} columns')
    print(f'Memory usage: {df.memory_usage(deep=True).sum() / 1e6:.2f} MB')
    print('\nColumn Types:')
    print(df.dtypes.value_counts())
    print('\nColumns:')
    print(df.columns.tolist())
    print('\nFirst 3 rows:')
    display(df.head(3))
    print('\n')

In [ ]:
# Explore Members dataset
explore_dataset(df_member, 'Members')

In [ ]:
# Explore Claims dataset
explore_dataset(df_claims, 'Claims')

In [ ]:
# Explore Calls dataset
explore_dataset(df_calls, 'Calls')

In [ ]:
# Explore Preauth dataset
explore_dataset(df_preauth, 'Preauth')

## 3. Missing Value Analysis

In [ ]:
def missing_value_analysis(df, name):
    """Analyze missing values in a dataframe."""
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    
    missing_df = pd.DataFrame({
        'Column': missing.index,
        'Missing Count': missing.values,
        'Missing %': missing_pct.values
    })
    missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing %', ascending=False)
    
    print(f'\n{name} - Missing Value Summary')
    print('=' * 50)
    if len(missing_df) == 0:
        print('No missing values found!')
    else:
        print(f'Columns with missing values: {len(missing_df)}')
        display(missing_df)
    
    return missing_df

In [ ]:
# Analyze missing values for all datasets
missing_member = missing_value_analysis(df_member, 'Members')
missing_claims = missing_value_analysis(df_claims, 'Claims')
missing_calls = missing_value_analysis(df_calls, 'Calls')
missing_preauth = missing_value_analysis(df_preauth, 'Preauth')

In [ ]:
# Visualize missing values
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

datasets = [
    (df_member, 'Members'),
    (df_claims, 'Claims'),
    (df_calls, 'Calls'),
    (df_preauth, 'Preauth')
]

for ax, (df, name) in zip(axes.flatten(), datasets):
    missing_pct = (df.isnull().sum() / len(df)) * 100
    missing_pct = missing_pct[missing_pct > 0].sort_values(ascending=True)
    
    if len(missing_pct) > 0:
        missing_pct.plot(kind='barh', ax=ax, color='coral')
        ax.set_xlabel('Missing %')
        ax.set_title(f'{name} - Missing Values')
    else:
        ax.text(0.5, 0.5, 'No missing values', ha='center', va='center', fontsize=12)
        ax.set_title(f'{name} - Missing Values')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'missing_values_overview.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Data Type Standardization and Cleaning

In [ ]:
# Standardize column names (uppercase, strip whitespace)
def standardize_columns(df):
    df.columns = df.columns.str.strip().str.upper()
    return df

df_member = standardize_columns(df_member)
df_claims = standardize_columns(df_claims)
df_calls = standardize_columns(df_calls)
df_preauth = standardize_columns(df_preauth)
df_provider = standardize_columns(df_provider)

print('Column names standardized.')

In [ ]:
# Check columns after standardization
print('Members columns:', df_member.columns.tolist())
print('\nClaims columns:', df_claims.columns.tolist())
print('\nCalls columns:', df_calls.columns.tolist())
print('\nPreauth columns:', df_preauth.columns.tolist())

In [ ]:
# Decode bytes columns if any exist
def decode_bytes_columns(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            # Check if values are bytes
            sample = df[col].dropna().head(1)
            if len(sample) > 0 and isinstance(sample.iloc[0], bytes):
                df[col] = df[col].apply(lambda x: x.decode('latin1') if isinstance(x, bytes) else x)
                print(f'Decoded column: {col}')
    return df

df_member = decode_bytes_columns(df_member)
df_claims = decode_bytes_columns(df_claims)
df_calls = decode_bytes_columns(df_calls)
df_preauth = decode_bytes_columns(df_preauth)

print('\nByte decoding complete.')

## 5. Key Statistics and Unique Values

In [ ]:
# Key statistics for Members
print('MEMBERS - Key Statistics')
print('=' * 50)
print(f"Unique Members (ADHERENT_NO): {df_member['ADHERENT_NO'].nunique():,}")
print(f"Unique Contracts (CONTRACT_NO): {df_member['CONTRACT_NO'].nunique():,}")
print(f"Unique Plans (PLAN_ID): {df_member['PLAN_ID'].nunique():,}")
print(f"Contract periods (CONT_YYMM): {df_member['CONT_YYMM'].nunique()}")
print(f"Date range: {df_member['CONT_YYMM'].min()} to {df_member['CONT_YYMM'].max()}")

In [ ]:
# Key statistics for Claims
print('CLAIMS - Key Statistics')
print('=' * 50)
print(f"Total claims rows: {len(df_claims):,}")
print(f"Unique Members: {df_claims['ADHERENT_NO'].nunique():,}")
print(f"Unique Contracts: {df_claims['CONT_NO'].nunique():,}")
print(f"Unique Providers: {df_claims['PROV_CODE'].nunique():,}")
print(f"Total Net Billed: {df_claims['SUM_OF_NETBILLED'].sum():,.2f}")
print(f"Claim Types: {df_claims['CLAIM_TYPE'].unique().tolist()}")

In [ ]:
# Key statistics for Calls
print('CALLS - Key Statistics')
print('=' * 50)
print(f"Total calls: {len(df_calls):,}")
print(f"Unique Contracts: {df_calls['CONT_NO'].nunique():,}")
print(f"Call Categories: {df_calls['CALL_CAT'].nunique()}")
print(f"\nCall Category Distribution:")
print(df_calls['CALL_CAT'].value_counts().head(10))

In [ ]:
# Key statistics for Preauth
print('PREAUTH - Key Statistics')
print('=' * 50)
print(f"Total preauth rows: {len(df_preauth):,}")
print(f"Unique Contracts: {df_preauth['CONT_NO'].nunique():,}")
print(f"Unique Episodes: {df_preauth['PREAUTH_EPISODE_ID'].nunique():,}")
print(f"Episode Status Distribution:")
print(df_preauth['EPISODE_STATUS'].value_counts())

## 6. Outlier Detection

In [ ]:
def detect_outliers_iqr(series, name):
    """Detect outliers using IQR method."""
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = series[(series < lower_bound) | (series > upper_bound)]
    
    print(f'\n{name} Outlier Analysis (IQR Method)')
    print('-' * 40)
    print(f'Q1: {Q1:,.2f}')
    print(f'Q3: {Q3:,.2f}')
    print(f'IQR: {IQR:,.2f}')
    print(f'Lower bound: {lower_bound:,.2f}')
    print(f'Upper bound: {upper_bound:,.2f}')
    print(f'Outliers count: {len(outliers):,} ({len(outliers)/len(series)*100:.2f}%)')
    
    return outliers, lower_bound, upper_bound

In [ ]:
# Analyze outliers in Net Billed Amount (Claims)
outliers_netbilled, lb_nb, ub_nb = detect_outliers_iqr(
    df_claims['SUM_OF_NETBILLED'].dropna(), 
    'Net Billed Amount'
)

# Distribution plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram (capped for visibility)
ax1 = axes[0]
data_capped = df_claims['SUM_OF_NETBILLED'].clip(upper=df_claims['SUM_OF_NETBILLED'].quantile(0.99))
ax1.hist(data_capped, bins=50, color='steelblue', edgecolor='white', alpha=0.7)
ax1.axvline(ub_nb, color='red', linestyle='--', label=f'Upper bound: {ub_nb:,.0f}')
ax1.set_xlabel('Net Billed Amount')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Net Billed Amount (99th percentile cap)')
ax1.legend()

# Box plot
ax2 = axes[1]
ax2.boxplot(data_capped, vert=True)
ax2.set_ylabel('Net Billed Amount')
ax2.set_title('Box Plot of Net Billed Amount')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'outliers_netbilled.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Analyze outliers in Estimated Amount (Preauth)
if 'EST_AMT' in df_preauth.columns:
    outliers_estamt, lb_ea, ub_ea = detect_outliers_iqr(
        df_preauth['EST_AMT'].dropna(), 
        'Estimated Amount (Preauth)'
    )

In [ ]:
# Analyze Written Premium outliers (Members)
if 'WP' in df_member.columns:
    outliers_wp, lb_wp, ub_wp = detect_outliers_iqr(
        df_member['WP'].dropna(), 
        'Written Premium'
    )

## 7. Data Aggregation at Contract Level

We will aggregate data at the **Contract (CONT_NO)** level since IVI is evaluated per corporate client.

In [ ]:
# Aggregate Members data by Contract
print('Aggregating Members data...')

agg_member = df_member.groupby('CONTRACT_NO').agg(
    total_members=('ADHERENT_NO', 'nunique'),
    total_plans=('PLAN_ID', 'nunique'),
    total_written_premium=('WP', 'sum'),
    total_earned=('WE', 'sum'),
    unique_networks=('PLAN_NETWORK', 'nunique'),
    unique_nationalities=('NATIONALITY', 'nunique'),
    male_count=('GENDER', lambda x: (x == 'M').sum() if x.dtype == 'object' else 0),
    female_count=('GENDER', lambda x: (x == 'F').sum() if x.dtype == 'object' else 0),
).reset_index()

agg_member.columns = ['CONTRACT_NO', 'TOTAL_MEMBERS', 'TOTAL_PLANS', 'TOTAL_WRITTEN_PREMIUM', 
                      'TOTAL_EARNED', 'UNIQUE_NETWORKS', 'UNIQUE_NATIONALITIES', 
                      'MALE_COUNT', 'FEMALE_COUNT']

print(f'Aggregated Members shape: {agg_member.shape}')
agg_member.head()

In [ ]:
# Aggregate Claims data by Contract
print('Aggregating Claims data...')

agg_claims = df_claims.groupby('CONT_NO').agg(
    total_claims=('VOU_NO', 'count'),
    unique_claims=('VOU_NO', 'nunique'),
    total_net_billed=('SUM_OF_NETBILLED', 'sum'),
    avg_net_billed=('SUM_OF_NETBILLED', 'mean'),
    max_net_billed=('SUM_OF_NETBILLED', 'max'),
    unique_members_with_claims=('ADHERENT_NO', 'nunique'),
    unique_providers=('PROV_CODE', 'nunique'),
    unique_diagnoses=('DIAG_CODE', 'nunique'),
    unique_benefit_heads=('BEN_HEAD', 'nunique'),
).reset_index()

agg_claims.columns = ['CONTRACT_NO', 'TOTAL_CLAIM_LINES', 'UNIQUE_CLAIMS', 'TOTAL_NET_BILLED',
                      'AVG_NET_BILLED', 'MAX_NET_BILLED', 'MEMBERS_WITH_CLAIMS',
                      'UNIQUE_PROVIDERS', 'UNIQUE_DIAGNOSES', 'UNIQUE_BENEFIT_HEADS']

print(f'Aggregated Claims shape: {agg_claims.shape}')
agg_claims.head()

In [ ]:
# Aggregate Calls data by Contract
print('Aggregating Calls data...')

agg_calls = df_calls.groupby('CONT_NO').agg(
    total_calls=('CALL_ID', 'count'),
    unique_calls=('CALL_ID', 'nunique'),
    unique_call_categories=('CALL_CAT', 'nunique'),
    unique_callers=('MBR_NO', 'nunique'),
).reset_index()

agg_calls.columns = ['CONTRACT_NO', 'TOTAL_CALLS', 'UNIQUE_CALLS', 
                     'UNIQUE_CALL_CATEGORIES', 'UNIQUE_CALLERS']

print(f'Aggregated Calls shape: {agg_calls.shape}')
agg_calls.head()

In [ ]:
# Aggregate Preauth data by Contract
print('Aggregating Preauth data...')

agg_preauth = df_preauth.groupby('CONT_NO').agg(
    total_preauth_items=('PREAUTH_EPISODE_ITEM_ID', 'count'),
    unique_episodes=('PREAUTH_EPISODE_ID', 'nunique'),
    unique_members_preauth=('MBR_NO', 'nunique'),
    total_estimated_amt=('EST_AMT', 'sum'),
    avg_estimated_amt=('EST_AMT', 'mean'),
    unique_providers_preauth=('PROV_CODE', 'nunique'),
).reset_index()

agg_preauth.columns = ['CONTRACT_NO', 'TOTAL_PREAUTH_ITEMS', 'UNIQUE_PREAUTH_EPISODES',
                       'MEMBERS_WITH_PREAUTH', 'TOTAL_ESTIMATED_AMT', 'AVG_ESTIMATED_AMT',
                       'UNIQUE_PROVIDERS_PREAUTH']

print(f'Aggregated Preauth shape: {agg_preauth.shape}')
agg_preauth.head()

In [ ]:
# Calculate preauth approval/rejection rates
print('Calculating preauth status rates...')

preauth_status = df_preauth.groupby(['CONT_NO', 'EPISODE_STATUS']).size().unstack(fill_value=0)
preauth_status['TOTAL'] = preauth_status.sum(axis=1)

# Calculate rates for common statuses
for col in preauth_status.columns:
    if col != 'TOTAL':
        preauth_status[f'{col}_RATE'] = preauth_status[col] / preauth_status['TOTAL']

preauth_status = preauth_status.reset_index()
preauth_status.columns = [str(c).upper().replace(' ', '_') for c in preauth_status.columns]
preauth_status = preauth_status.rename(columns={'CONT_NO': 'CONTRACT_NO'})

print(f'Preauth status rates shape: {preauth_status.shape}')
preauth_status.head()

In [ ]:
# Calculate call category distribution per contract
print('Calculating call category distribution...')

call_cats = df_calls.groupby(['CONT_NO', 'CALL_CAT']).size().unstack(fill_value=0)
call_cats = call_cats.add_prefix('CALLS_')
call_cats = call_cats.reset_index()
call_cats.columns = [str(c).upper().replace(' ', '_') for c in call_cats.columns]
call_cats = call_cats.rename(columns={'CONT_NO': 'CONTRACT_NO'})

print(f'Call categories shape: {call_cats.shape}')
call_cats.head()

## 8. Merge All Aggregated Data

In [ ]:
# Merge all aggregated datasets
print('Merging all aggregated datasets...')

# Start with members (base)
df_contract = agg_member.copy()
print(f'After members: {df_contract.shape}')

# Merge claims
df_contract = df_contract.merge(agg_claims, on='CONTRACT_NO', how='left')
print(f'After claims: {df_contract.shape}')

# Merge calls
df_contract = df_contract.merge(agg_calls, on='CONTRACT_NO', how='left')
print(f'After calls: {df_contract.shape}')

# Merge preauth
df_contract = df_contract.merge(agg_preauth, on='CONTRACT_NO', how='left')
print(f'After preauth: {df_contract.shape}')

# Merge preauth status rates (select key columns)
status_cols = ['CONTRACT_NO'] + [c for c in preauth_status.columns if '_RATE' in c]
df_contract = df_contract.merge(preauth_status[status_cols], on='CONTRACT_NO', how='left')
print(f'After preauth status: {df_contract.shape}')

print(f'\nFinal merged dataset shape: {df_contract.shape}')

In [ ]:
# View the merged dataset
df_contract.head()

In [ ]:
# Check for any remaining missing values after merge
missing_after_merge = df_contract.isnull().sum()
missing_after_merge = missing_after_merge[missing_after_merge > 0]

print('Missing values after merge:')
print(missing_after_merge)

## 9. Create Derived Features (IVI Components)

In [ ]:
# Create derived features for IVI calculation
print('Creating derived features...')

# Utilization metrics
df_contract['UTILIZATION_RATE'] = df_contract['MEMBERS_WITH_CLAIMS'] / df_contract['TOTAL_MEMBERS']
df_contract['CLAIMS_PER_MEMBER'] = df_contract['TOTAL_CLAIM_LINES'] / df_contract['TOTAL_MEMBERS']
df_contract['COST_PER_MEMBER'] = df_contract['TOTAL_NET_BILLED'] / df_contract['TOTAL_MEMBERS']
df_contract['COST_PER_UTILIZER'] = df_contract['TOTAL_NET_BILLED'] / df_contract['MEMBERS_WITH_CLAIMS'].replace(0, np.nan)

# Loss ratio (key sustainability metric)
df_contract['LOSS_RATIO'] = df_contract['TOTAL_NET_BILLED'] / df_contract['TOTAL_EARNED'].replace(0, np.nan)

# Experience metrics
df_contract['CALLS_PER_MEMBER'] = df_contract['TOTAL_CALLS'] / df_contract['TOTAL_MEMBERS']
df_contract['PREAUTH_PER_MEMBER'] = df_contract['UNIQUE_PREAUTH_EPISODES'] / df_contract['TOTAL_MEMBERS']

# Gender ratio
df_contract['MALE_RATIO'] = df_contract['MALE_COUNT'] / df_contract['TOTAL_MEMBERS']

print('Derived features created.')
print(f'\nNew columns: {[c for c in df_contract.columns if c not in agg_member.columns]}')

In [ ]:
# Fill NaN values with appropriate defaults
print('Handling missing values in derived features...')

# For contracts with no claims/calls/preauth, fill with 0
fill_zero_cols = ['TOTAL_CLAIM_LINES', 'UNIQUE_CLAIMS', 'TOTAL_NET_BILLED', 'AVG_NET_BILLED',
                  'MAX_NET_BILLED', 'MEMBERS_WITH_CLAIMS', 'UNIQUE_PROVIDERS', 'UNIQUE_DIAGNOSES',
                  'UNIQUE_BENEFIT_HEADS', 'TOTAL_CALLS', 'UNIQUE_CALLS', 'UNIQUE_CALL_CATEGORIES',
                  'UNIQUE_CALLERS', 'TOTAL_PREAUTH_ITEMS', 'UNIQUE_PREAUTH_EPISODES',
                  'MEMBERS_WITH_PREAUTH', 'TOTAL_ESTIMATED_AMT', 'AVG_ESTIMATED_AMT',
                  'UNIQUE_PROVIDERS_PREAUTH', 'UTILIZATION_RATE', 'CLAIMS_PER_MEMBER',
                  'COST_PER_MEMBER', 'CALLS_PER_MEMBER', 'PREAUTH_PER_MEMBER']

for col in fill_zero_cols:
    if col in df_contract.columns:
        df_contract[col] = df_contract[col].fillna(0)

# Fill rate columns with 0
rate_cols = [c for c in df_contract.columns if '_RATE' in c]
for col in rate_cols:
    df_contract[col] = df_contract[col].fillna(0)

# Check remaining missing
remaining_missing = df_contract.isnull().sum()
remaining_missing = remaining_missing[remaining_missing > 0]
print(f'\nRemaining missing values: {len(remaining_missing)}')
if len(remaining_missing) > 0:
    print(remaining_missing)

## 10. Final Dataset Summary and Export

In [ ]:
# Final dataset summary
print('FINAL DATASET SUMMARY')
print('=' * 60)
print(f'Shape: {df_contract.shape[0]:,} contracts x {df_contract.shape[1]} features')
print(f'\nMemory usage: {df_contract.memory_usage(deep=True).sum() / 1e6:.2f} MB')
print(f'\nColumn types:')
print(df_contract.dtypes.value_counts())
print(f'\nMissing values: {df_contract.isnull().sum().sum()}')

In [ ]:
# Descriptive statistics
df_contract.describe()

In [ ]:
# All columns in final dataset
print('All columns in final dataset:')
for i, col in enumerate(df_contract.columns, 1):
    print(f'{i:2}. {col}')

In [ ]:
# Save the cleaned dataset
output_file = OUTPUT_DIR / 'contract_aggregated_data.parquet'
df_contract.to_parquet(output_file, index=False)
print(f'Dataset saved to: {output_file}')

# Also save as CSV for easy inspection
csv_file = OUTPUT_DIR / 'contract_aggregated_data.csv'
df_contract.to_csv(csv_file, index=False)
print(f'CSV backup saved to: {csv_file}')

In [ ]:
# Visualize key distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Contract size distribution
axes[0, 0].hist(df_contract['TOTAL_MEMBERS'].clip(upper=df_contract['TOTAL_MEMBERS'].quantile(0.99)), 
                bins=50, color='steelblue', edgecolor='white')
axes[0, 0].set_xlabel('Total Members')
axes[0, 0].set_title('Contract Size Distribution')

# Loss ratio distribution
axes[0, 1].hist(df_contract['LOSS_RATIO'].dropna().clip(upper=3), 
                bins=50, color='coral', edgecolor='white')
axes[0, 1].axvline(1.0, color='red', linestyle='--', label='Break-even (100%)')
axes[0, 1].set_xlabel('Loss Ratio')
axes[0, 1].set_title('Loss Ratio Distribution')
axes[0, 1].legend()

# Utilization rate
axes[0, 2].hist(df_contract['UTILIZATION_RATE'].dropna(), 
                bins=50, color='green', edgecolor='white')
axes[0, 2].set_xlabel('Utilization Rate')
axes[0, 2].set_title('Member Utilization Rate')

# Cost per member
axes[1, 0].hist(df_contract['COST_PER_MEMBER'].clip(upper=df_contract['COST_PER_MEMBER'].quantile(0.99)), 
                bins=50, color='purple', edgecolor='white')
axes[1, 0].set_xlabel('Cost per Member')
axes[1, 0].set_title('Cost per Member Distribution')

# Calls per member
axes[1, 1].hist(df_contract['CALLS_PER_MEMBER'].clip(upper=df_contract['CALLS_PER_MEMBER'].quantile(0.99)), 
                bins=50, color='orange', edgecolor='white')
axes[1, 1].set_xlabel('Calls per Member')
axes[1, 1].set_title('Calls per Member Distribution')

# Written premium distribution
axes[1, 2].hist(df_contract['TOTAL_WRITTEN_PREMIUM'].clip(upper=df_contract['TOTAL_WRITTEN_PREMIUM'].quantile(0.99)), 
                bins=50, color='teal', edgecolor='white')
axes[1, 2].set_xlabel('Total Written Premium')
axes[1, 2].set_title('Written Premium Distribution')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'contract_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Correlation heatmap for key metrics
key_metrics = ['TOTAL_MEMBERS', 'TOTAL_WRITTEN_PREMIUM', 'TOTAL_NET_BILLED', 'LOSS_RATIO',
               'UTILIZATION_RATE', 'COST_PER_MEMBER', 'CLAIMS_PER_MEMBER', 'CALLS_PER_MEMBER',
               'PREAUTH_PER_MEMBER']

corr_matrix = df_contract[key_metrics].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0, 
            fmt='.2f', square=True, linewidths=0.5)
plt.title('Correlation Matrix - Key Contract Metrics')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

## Summary

### Data Processing Completed:
1. Loaded 4 SAS datasets (members, claims, calls, preauth)
2. Standardized column names and decoded byte strings
3. Analyzed missing values across all datasets
4. Detected outliers using IQR method
5. Aggregated all data at contract level
6. Created derived features for IVI calculation
7. Saved cleaned dataset to parquet and CSV formats

### Output Files:
- `data/processed/contract_aggregated_data.parquet`
- `data/processed/contract_aggregated_data.csv`
- `data/processed/missing_values_overview.png`
- `data/processed/outliers_netbilled.png`
- `data/processed/contract_distributions.png`
- `data/processed/correlation_matrix.png`